<a href="https://colab.research.google.com/github/zhangxun-lime/sts/blob/master/R_heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1、矩形热图绘制代码 
## Draw a rectangular heatmap


In [0]:
# 加载热图需要的函数包
install.packages("pheatmap")
install.packages("vegan")
install.packages("gplots")
library(pheatmap)
library(gplots)
library(vegan)

In [0]:
## 读取csv文件
setwd("/content/heatmap-rectangular")
getwd()
myfiles <- list.files(pattern = "*.csv")
myfiles
matrix <- read.csv(myfiles[1], sep=',', header=T, check.names=FALSE, row.names=1)
head(matrix)
## 提取xx表达值所在的列，组成新的矩阵，并将矩阵转置
## 由于 R 语言的 scale 函数是按列归一化，对于我们一般习惯基因名为行，样本名为列的数据框，就需要进行转置
mat <- t(matrix[,1:45])
head(mat)
## 归一化处理
mat <- scale(mat, center = TRUE, scale = TRUE)
head(mat)
## 绘制热图
pheatmap(mat)
## 绘制热图(调试参数)
# pheatmap(mat,cellwidth = NA, cellheight = NA, 
#         treeheight_row = 50, treeheight_col = 50 ,color = colorRampPalette(c("blue", "white","red"))(100), 
#         scale ="row", legend = TRUE,border_color = NA, fontsize_row = 8, fontsize_col = 10,
#         clustering_distance_rows = drows, clustering_distance_cols = dcols, clustering_method ="average",main ="Heatmap")
dev.off()

#2、环状热图绘制代码
## Draw a ring heatmap




In [0]:
# 加载热图需要的函数包
install.packages("dendextend")
install.packages("circlize")
library(dendextend)
require("circlize")
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("ComplexHeatmap")
library(ComplexHeatmap)

In [0]:
# 1、读取csv文件中的实验数据，对xx表达量归一化处理
setwd("/content/heatmap")
myfiles <- list.files(pattern = "*.csv")
matrix <- read.csv(myfiles[1], sep=',', header=T, check.names=FALSE, row.names=1)
mat <- t(matrix[,1:15])
mat <- scale(mat, center = TRUE, scale = TRUE)
# 2、对数据进行聚类
dend <- as.dendrogram(hclust(dist(t(mat))))
n <- 3;dend <- dend %>% set("branches_k_color", k = n) 
par(mar=c(7.5,3,1,0));plot(dend)
mat2 <- mat[, order.dendrogram(dend)]
lable1 <- row.names(mat2);lable2 <- colnames(mat2);
nr <- nrow(mat2);nr;nc <- ncol(mat2);nc
# 3、将xx表达量数值转变为颜色
col_fun <- colorRamp2(c(-1.5, 0, 1.5), c("deepskyblue", "white", "red"))
col_mat <- col_fun(mat2);col_mat[,1];
# 4、画板初始化设置
par(mar <- c(0,0,0,0));circos.clear();
circos.par(canvas.xlim = c(-1.4,1.4),canvas.ylim = c(-1.4,1.4),cell.padding = c(0,0,0,0),gap.degree = 90)
factors <- "a"
circos.initialize(factors, xlim = c(0, ncol(mat2)))
# 5、构建输出图像
circos.track(ylim = c(0, nr),bg.border = NA,track.height = 0.05*nr,
        panel.fun = function(x, y) {
          for(i in 1:nr) {
            circos.rect(xleft = 1:nc - 1, ybottom = rep(nr - i, nc),xright = 1:nc, 
            ytop = rep(nr - i + 1, nc),border = "white",col = col_mat[i,])
            circos.text(x = nc,y = 15.2 -i,labels = lable1[i],facing = "downward", 
            niceFacing = TRUE,cex = 0.6,adj = c(-0.2, 0))        
}})
for(i in 1:nc){
  circos.text(x = i-0.4,y = 16,labels = lable2[i],
              facing = "clockwise", niceFacing = TRUE,cex = 0.5,adj = c(0, 0))
}
max_height <-max(attr(dend, "height"))
circos.track(ylim = c(0, max_height),bg.border = NA,track.height = 0.15,
             panel.fun = function(x, y){circos.dendrogram(dend = dend,max_height = max_height)
})
circos.clear()
lgd <- Legend(at = c(-2,-1, 0, 1, 2), col_fun = col_fun,title_position = "topcenter",title = "Z-score")
draw(lgd, x = unit(0.75, "npc"), y = unit(0.75, "npc"))